In [1]:
from tensorflow import keras
from tensorflow.keras.utils import get_file, load_img, img_to_array, save_img
from tensorflow.keras.applications.vgg19 import preprocess_input, VGG19
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import numpy as np
import tensorflow as tf

In [2]:
base_image_path = get_file("sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path = get_file("starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")
original_width, original_height = load_img(base_image_path).size

img_height = 512
img_width = round(original_width * img_height / original_height)

**Auxiliary functions**

In [3]:
def preprocess_image(image_path):
    
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

def deprocess_image(img):
    
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    
    return img

**Using a pretrained VGG19 model to create a feature extractor**

In [4]:
model = VGG19(weights="imagenet", include_top=False)

In [5]:
outputs = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs)

**Content loss**

In [6]:
def content_loss(base_img, combination_img):
    
    return tf.reduce_sum(tf.square(combination_img - base_img))

**Style loss**

In [7]:
def gram_matrix(x):
    
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    
    return gram

def style_loss(style_img, combination_img):
    
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

**Total variation loss**

In [8]:
def total_variation_loss(x):
    
    a = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :])
    b = tf.square(x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :])
    
    return tf.reduce_sum(tf.pow(a + b, 1.25))

**Defining the final loss that you'll minimize**

In [9]:
style_layers = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]

content_layer_name = "block5_conv2"

total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

In [10]:
def compute_loss(combination_image, base_image, style_reference_image):
    
    input_tensor = tf.concat([base_image, style_reference_image, combination_image], axis=0)
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(base_image_features, combination_features)
    
    for layer in style_layers:
        
        layer_features = features[layer]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(style_reference_features, combination_features)
        loss += (style_weight / len(style_layers)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    
    return loss

**Setting up the gradient-descent process**

In [11]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    
    with tf.GradientTape() as tape:
        
        loss = compute_loss(combination_image, base_image, style_reference_image)
    
    grads = tape.gradient(loss, combination_image)
    
    return loss, grads

In [12]:
optimizer = SGD(ExponentialDecay(initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96))

In [13]:
base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

for i in range(1, 4001):
    
    loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)
    optimizer.apply_gradients([(grads, combination_image)])
    
    if i % 100 == 0:
        
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        
        if (i % 1000 == 0):
            save_img(fname, img)

Iteration 100: loss=11061.29
Iteration 200: loss=9080.97
Iteration 300: loss=8241.46
Iteration 400: loss=7767.04
Iteration 500: loss=7457.57
Iteration 600: loss=7236.77
Iteration 700: loss=7069.80
Iteration 800: loss=6938.43
Iteration 900: loss=6831.84
Iteration 1000: loss=6743.32
Iteration 1100: loss=6668.53
Iteration 1200: loss=6604.63
Iteration 1300: loss=6549.45
Iteration 1400: loss=6501.16
Iteration 1500: loss=6458.66
Iteration 1600: loss=6420.88
Iteration 1700: loss=6387.18
Iteration 1800: loss=6356.83
Iteration 1900: loss=6329.47
Iteration 2000: loss=6304.64
Iteration 2100: loss=6282.06
Iteration 2200: loss=6261.39
Iteration 2300: loss=6242.47
Iteration 2400: loss=6225.08
Iteration 2500: loss=6209.06
Iteration 2600: loss=6194.28
Iteration 2700: loss=6180.63
Iteration 2800: loss=6168.01
Iteration 2900: loss=6156.28
Iteration 3000: loss=6145.38
Iteration 3100: loss=6135.23
Iteration 3200: loss=6125.73
Iteration 3300: loss=6116.85
Iteration 3400: loss=6108.52
Iteration 3500: loss=6